# 수정 포인트
1. 데이터 증강 추가, 기존에는 정규화만 있었음, resnet18 백본 사용
2. Early Stopping, resnet18 백본 사용
3. ConvNeXt-Tiny로 백본 변경

# Import

In [1]:
import os
import random

import pandas as pd
import numpy as np

from PIL import Image
from tqdm import tqdm 

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn, optim
import torch.nn as nn

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import log_loss
import torch.nn as nn
import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

c:\Users\os415\.conda\envs\hai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


# Hyperparameter Setting

In [2]:
# 'IMG_SIZE': 224, # 모델에 입력되는 이미지 크기
# 'BATCH_SIZE': 64, # 한 번에 모델에 넣는 이미지 수
# 'EPOCHS': 50, # 전체 데이터를 몇 번 반복 학습할지
#'LEARNING_RATE': 가중치 업데이트 정도를 조절하는 학습률
# 'SEED' : 무작위 요소들을 고정하여 실험 결과를 재현 가능하게 함
# 
CFG = {
    'IMG_SIZE': 224,
    'BATCH_SIZE': 64,
    'EPOCHS': 50,
    'LEARNING_RATE': 1e-4,
    'SEED' : 42
}

# Fixed RandomSeed

In [3]:
# 딥러닝 학습 과정에서는 무작위성이 많이 개입된
# 무작위성: 데이터 로딩 순서, 가중치 초기화, dropout, 데이터 증강, gpu 연산의 비결정성
# 그레서 시드를 고정하지 안으면 매번 실행할 때마다 결과가 달라짐
# 동일한 결과가 재현되도록 하기 위한 설정이 seed_everyting 함수수
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

# CustomDataset

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None, is_test=False):
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test
        self.samples = []

        if is_test:
            # 테스트셋: 라벨 없이 이미지 경로만 저장
            for fname in sorted(os.listdir(root_dir)):
                if fname.lower().endswith(('.jpg')):
                    img_path = os.path.join(root_dir, fname)
                    self.samples.append((img_path,))
        else:
            # 학습셋: 클래스별 폴더 구조에서 라벨 추출
            self.classes = sorted(os.listdir(root_dir))
            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

            for cls_name in self.classes:
                cls_folder = os.path.join(root_dir, cls_name)
                for fname in os.listdir(cls_folder):
                    if fname.lower().endswith(('.jpg')):
                        img_path = os.path.join(cls_folder, fname)
                        label = self.class_to_idx[cls_name]
                        self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if self.is_test:
            img_path = self.samples[idx][0]
            image = np.array(Image.open(img_path).convert('RGB'))
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        else:
            img_path, label = self.samples[idx]
            image = np.array(Image.open(img_path).convert('RGB'))
            if self.transform:
                image = self.transform(image=image)['image']
            return image, label


# Data Load

In [5]:
train_root = './data/train'
test_root = './data/test'

In [6]:
# 학습 데이터에 적용할 데이터 증강 파이프라인
train_transform = A.Compose([
    # 이미지 크기를 모델 입력 크기에 맞게 고정
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),

    # 좌우 반전: 차량이 좌우 어느 방향을 향해 있어도 학습 가능하도록
    A.HorizontalFlip(p=0.5),

    # 밝기/대비 랜덤 조절: 야외 촬영 시 조명 차이를 반영
    A.RandomBrightnessContrast(p=0.2),

    # 색조(Hue), 채도(Saturation), 명도(Value) 변경: 색상에 덜 민감하게
    A.HueSaturationValue(p=0.2),

    # 이미지 이동, 확대/축소, 회전: 다양한 촬영 각도 및 위치 대응
    A.ShiftScaleRotate(shift_limit=0.05,   # 최대 ±5% 이동
                       scale_limit=0.05,   # 최대 ±5% 확대/축소
                       rotate_limit=15,    # 최대 ±15도 회전
                       p=0.5),

    # 그림자 효과 추가: 자연광 환경에서의 촬영 상황을 반영
    A.RandomShadow(p=0.2),

    # RGB 채널별 색상 이동: 다양한 카메라 환경, 화이트밸런스 차이 대응
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.2),

    # 이미지 일부분을 무작위로 지우기 (일부 가림 상황을 반영)
    A.CoarseDropout(max_holes=2,          # 최대 2개 영역 제거
                    max_height=16, 
                    max_width=16, 
                    p=0.3),

    # 픽셀값을 정규화 (ImageNet 사전학습 모델 기준 평균/표준편차)
    A.Normalize(mean=(0.485, 0.456, 0.406), 
                std=(0.229, 0.224, 0.225)),

    # numpy 이미지 → PyTorch Tensor 변환
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), 
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

c:\Users\os415\.conda\envs\hai\Lib\site-packages\albumentations\core\validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\os415\AppData\Local\Temp\ipykernel_29460\2295189183.py:28: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=2,          # 최대 2개 영역 제거


In [7]:
# 전체 데이터셋 로드
full_dataset = CustomImageDataset(train_root, transform=None)
print(f"총 이미지 수: {len(full_dataset)}")

targets = [label for _, label in full_dataset.samples]
class_names = full_dataset.classes

# Stratified Split
train_idx, val_idx = train_test_split(
    range(len(targets)), test_size=0.2, stratify=targets, random_state=42
)

# Subset + transform 각각 적용
train_dataset = Subset(CustomImageDataset(train_root, transform=train_transform), train_idx)
val_dataset = Subset(CustomImageDataset(train_root, transform=val_transform), val_idx)
print(f'train 이미지 수: {len(train_dataset)}, valid 이미지 수: {len(val_dataset)}')


# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

총 이미지 수: 33137
train 이미지 수: 26509, valid 이미지 수: 6628


# Model Define

In [8]:
class BaseModel(nn.Module):
    def __init__(self, model_name: str, num_classes: int):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model(model_name, pretrained=True, num_classes=0)  # 분류기 제거

        # 자동 in_features 추출
        try:
            in_features = self.backbone.num_features  # timm 공통 속성
        except:
            raise ValueError(f"Could not find in_features for model {model_name}")

        # pooling 여부 결정 (ConvNeXt 같은 경우 필요)
        self.needs_pooling = hasattr(self.backbone, 'head') and isinstance(self.backbone.head, nn.Identity) is False
        self.pool = nn.AdaptiveAvgPool2d((1, 1)) if self._is_2d_output() else None

        self.classifier = nn.Linear(in_features, num_classes)

    def _is_2d_output(self):
        # ConvNeXt, EfficientNet 등은 (B, C, H, W)로 출력됨 → AdaptiveAvgPool 필요
        example_input = torch.randn(1, 3, 224, 224)
        with torch.no_grad():
            output = self.backbone(example_input)
        return output.dim() == 4

    def forward(self, x):
        x = self.backbone(x)
        if self.pool:
            x = self.pool(x)
            x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [9]:
"""
# resnet18, 1번과 2번
class BaseModel(nn.Module):
    def __init__(self, num_classes):
        super(BaseModel, self).__init__()
        self.backbone = models.resnet18(pretrained=True)  # ResNet18 모델 불러오기
        self.feature_dim = self.backbone.fc.in_features 
        self.backbone.fc = nn.Identity()  # feature extractor로만 사용
        self.head = nn.Linear(self.feature_dim, num_classes)  # 분류기

    def forward(self, x):
        x = self.backbone(x)       
        x = self.head(x) 
        return x
"""

'\n# resnet18, 1번과 2번\nclass BaseModel(nn.Module):\n    def __init__(self, num_classes):\n        super(BaseModel, self).__init__()\n        self.backbone = models.resnet18(pretrained=True)  # ResNet18 모델 불러오기\n        self.feature_dim = self.backbone.fc.in_features \n        self.backbone.fc = nn.Identity()  # feature extractor로만 사용\n        self.head = nn.Linear(self.feature_dim, num_classes)  # 분류기\n\n    def forward(self, x):\n        x = self.backbone(x)       \n        x = self.head(x) \n        return x\n'

# Train/ Validation

In [10]:
model = BaseModel(model_name='efficientnetv2_rw_s', num_classes=len(class_names)).to(device)
best_logloss = float('inf')
# 추가: Early Stopping을 위한 변수
patience = 5  # 개선 없을 때 몇 epoch까지 기다릴지
counter = 0   # 현재까지 개선되지 않은 횟수

# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])

# 학습 및 검증 루프
for epoch in range(CFG['EPOCHS']):
    # Train
    model.train()
    train_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}/{CFG['EPOCHS']}] Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)  # logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"[Epoch {epoch+1}/{CFG['EPOCHS']}] Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Accuracy
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # LogLoss
            probs = F.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    val_logloss = log_loss(all_labels, all_probs, labels=list(range(len(class_names))))

    # 결과 출력
    print(f"Train Loss : {avg_train_loss:.4f} || Valid Loss : {avg_val_loss:.4f} | Valid Accuracy : {val_accuracy:.4f}%")

    # Best model 저장
    if val_logloss < best_logloss:
        best_logloss = val_logloss
        torch.save(model.state_dict(), f'best_model.pth')
        print(f"📦 Best model saved at epoch {epoch+1} (logloss: {val_logloss:.4f})")
        counter = 0  # 성능 개선되었으므로 초기화
    else:
        counter += 1
        print(f"⚠️ No improvement for {counter} epoch(s).")
        if counter >= patience:
            print(f"⏹ Early stopping triggered at epoch {epoch+1}")
            break  # 학습 중단

[Epoch 1/50] Validation: 100%|██████████| 104/104 [00:50<00:00,  2.07it/s]
c:\Users\os415\.conda\envs\hai\Lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss : 4.5783 || Valid Loss : 0.9014 | Valid Accuracy : 75.9656%
📦 Best model saved at epoch 1 (logloss: 0.9019)


[Epoch 2/50] Validation: 100%|██████████| 104/104 [00:45<00:00,  2.30it/s]
c:\Users\os415\.conda\envs\hai\Lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss : 0.4583 || Valid Loss : 0.2911 | Valid Accuracy : 90.8570%
📦 Best model saved at epoch 2 (logloss: 0.2916)


[Epoch 3/50] Validation: 100%|██████████| 104/104 [00:45<00:00,  2.30it/s]
c:\Users\os415\.conda\envs\hai\Lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


...

[Epoch 18/50] Validation: 100%|██████████| 104/104 [01:06<00:00,  1.56it/s]
c:\Users\os415\.conda\envs\hai\Lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss : 0.0553 || Valid Loss : 0.1772 | Valid Accuracy : 95.6548%
⚠️ No improvement for 4 epoch(s).


[Epoch 19/50] Validation: 100%|██████████| 104/104 [01:02<00:00,  1.67it/s]

Train Loss : 0.0484 || Valid Loss : 0.1704 | Valid Accuracy : 95.7453%
⚠️ No improvement for 5 epoch(s).
⏹ Early stopping triggered at epoch 19



c:\Users\os415\.conda\envs\hai\Lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


# Inference

In [11]:
test_dataset = CustomImageDataset(test_root, transform=val_transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [12]:
# 저장된 모델 로드
model = BaseModel(model_name='efficientnetv2_rw_s', num_classes=len(class_names))
model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.to(device)

# 추론
model.eval()
results = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)

        # 각 배치의 확률을 리스트로 변환
        for prob in probs.cpu():  # prob: (num_classes,)
            result = {
                class_names[i]: prob[i].item()
                for i in range(len(class_names))
            }
            results.append(result)
            
pred = pd.DataFrame(results)

# Submission

In [13]:
submission = pd.read_csv('./result/sample_submission.csv', encoding='utf-8-sig')

# 'ID' 컬럼을 제외한 클래스 컬럼 정렬
class_columns = submission.columns[1:]
pred = pred[class_columns]

submission[class_columns] = pred.values
submission.to_csv('./result/submission_04.csv', index=False, encoding='utf-8-sig')